In [2]:
import dataclasses
from typing import List, Optional

from pydantic import Field, TypeAdapter,ValidationError
from pydantic.dataclasses import dataclass


@dataclass
class User:
    id: int
    name: str = 'John Doe'
    friends: List[int] = dataclasses.field(default_factory=lambda: [0])
    datasets: List[str] = dataclasses.field(default_factory=lambda: ['swot'])
    age: Optional[int] = dataclasses.field(
        default=None,
        metadata=dict(title='The age of the user', description='do not lie!'),
    )
    height: Optional[int] = Field(None, title='The height in cm', ge=50, le=300)

print(TypeAdapter(User).json_schema())

{'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'default': 'John Doe', 'title': 'Name', 'type': 'string'}, 'friends': {'items': {'type': 'integer'}, 'title': 'Friends', 'type': 'array'}, 'datasets': {'items': {'type': 'string'}, 'title': 'Datasets', 'type': 'array'}, 'age': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'default': None, 'description': 'do not lie!', 'title': 'The age of the user'}, 'height': {'anyOf': [{'maximum': 300, 'minimum': 50, 'type': 'integer'}, {'type': 'null'}], 'default': None, 'title': 'The height in cm'}}, 'required': ['id'], 'title': 'User', 'type': 'object'}


In [3]:
try:
    user = User(id='42')
    print(user)
except pydantic.ValidationError as e:
        print(e)

User(id=42, name='John Doe', friends=[0], datasets=['swot'], age=None, height=None)


In [5]:
try:
    user = User(id='42',datasets=['swit'])
    print(user)
except pydantic.ValidationError as e:
        print(e)

User(id=42, name='John Doe', friends=[0], datasets=['swit'], age=None, height=None)


In [6]:
try:
    user = User(id='42',datasets=99)
    print(user)
except ValidationError as e:
        print(e)

1 validation error for User
datasets
  Input should be a valid list [type=list_type, input_value=99, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/list_type


In [7]:
try:
    user = User(id='42', height=3054)
    print(user)
except ValidationError as e:
        print(e)

1 validation error for User
height
  Input should be less than or equal to 300 [type=less_than_equal, input_value=3054, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/less_than_equal


In [9]:
from datetime import date
from uuid import UUID, uuid4
from enum import Enum
from pydantic import BaseModel

class Department(Enum):
    HR = "HR"
    SALES = "SALES"
    IT = "IT"
    ENGINEERING = "ENGINEERING"

class Employee(BaseModel):
    employee_id: UUID = uuid4()
    name: str
    date_of_birth: date
    salary: float
    department: Department
    elected_benefits: bool

In [13]:
Employee(
...     name="Chris DeTuma",
...     email="cdetuma@example.com",
...     date_of_birth="1998-04-02",
...     salary=123_000.00,
...     department="SHITIT",
...     elected_benefits=True,
)

ValidationError: 1 validation error for Employee
department
  Input should be 'HR', 'SALES', 'IT' or 'ENGINEERING' [type=enum, input_value='SHITIT', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/enum